In [ ]:
import cv2 as cv
import numpy as np
from open3d import *

In [ ]:
fx = 718.856; fy = 718.856; cx = 607.1928; cy = 185.2157
b = 0.573  # baseline
left = cv.imread('./left.png', cv.IMREAD_GRAYSCALE)
right = cv.imread('./right.png', cv.IMREAD_GRAYSCALE)
# Using Semi-global batch matching, it's "sensitve to parameters"
sgbm = cv.StereoSGBM_create(0, 96, 9, 8*9*9, 32*9*9, 1, 63, 10, 100, 32)
disparity = sgbm.compute(left, right)
disparity = disparity.astype('float32')*1/16


In [ ]:
pointcloud = []
for v in range(left.shape[0]):
    for u in range(left.shape[1]):
        if not(disparity.item(v, u) <= 10.0 or disparity.item(v, u) >= 96.0):
            x = (u - cx)/fx
            y = (v - cy)/fy
            depth = fx*b/disparity.item(v, u)
            point = np.array([x*depth, y*depth, depth, left.item(v, u)/255.0])
            pointcloud.append(point)

In [ ]:
cv.imshow("disparity", disparity/96)
cv.imshow("left", left)
cv.waitKey(0)

In [ ]:
points = []
colors = []
for point in pointcloud:
    points.append(point[:3])
    colors.append([point[3]]*3)
pcd = geometry.PointCloud()
pcd.points = utility.Vector3dVector(points)
pcd.colors = utility.Vector3dVector(colors)
visualization.draw_geometries([pcd])